# Get Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import json
import os.path

In [ ]:
t_map = pd.read_csv('/content/drive/MyDrive/CSVs/OpenStreetMap_Tourist_Attractions_for_North_America.csv')

<ipython-input-3-0ee21db77ba6>:1: DtypeWarning: Columns (4,7,10,15,18,23,35) have mixed types. Specify dtype option on import or set low_memory=False.
  t_map = pd.read_csv('/content/drive/MyDrive/CSVs/OpenStreetMap_Tourist_Attractions_for_North_America.csv')


In [ ]:
bcols = ['business_id', 'name', 'address', 'review_count', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'is_open', 'categories']
data = []
filename = '/content/drive/MyDrive/yelp_dataset/yelp_academic_dataset_business.json'

with open(filename) as f:
  for line in f:
    doc = json.loads(line)
    add = []
    for bcol in bcols:
      add.append(doc[bcol])
    data.append(add)

businesses = pd.DataFrame(data = data, columns = bcols)

In [ ]:
rcols = ['review_id', 'business_id', 'stars', 'text']
data = []
filename = '/content/drive/MyDrive/yelp_dataset/yelp_academic_dataset_review.json'

with open(filename) as f:
  for line in f:
    doc = json.loads(line)
    add = []
    for rcol in rcols:
      add.append(doc[rcol])
    data.append(add)

reviews = pd.DataFrame(data = data, columns = rcols)

# Business Preprocessing

## Standard Fare

In [ ]:
#Isolate to restaurants only

businesses['categories'] = businesses['categories'].fillna('Nothing')
restaurants = businesses

In [ ]:
#get categories
food_cats = ['Acai Bowls', 'Bagels', 'Bakeries', 'Bento', 'Breweries', 'Brewpubs', 'Bubble']
cats = restaurants['categories'].str.split(',', expand = True)

restaurants['cat_1'] = cats[1].str.strip()
restaurants['cat_2'] = cats[2].str.strip()
restaurants['cat_3'] = cats[3].str.strip()
restaurants = restaurants.drop('categories', axis = 1)

In [ ]:
#Drop closed businesses
restaurants = restaurants[restaurants['is_open'] == 1]
restaurants = restaurants.drop('is_open', axis = 1)

In [ ]:
#Unify addresses
restaurants['address'] = restaurants['address'] + ', ' + restaurants['city'] + ', ' + restaurants['state'] + ' ' + restaurants['postal_code']

In [ ]:
restaurants = restaurants.drop(['city', 'state'], axis = 1)
restaurants = restaurants.reset_index(drop = True)
restaurants

,business_id,name,address,review_count,postal_code,latitude,longitude,stars,cat_1,cat_2,cat_3
0,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,"87 Grasso Plaza Shopping Center, Affton, MO 63123",15,63123,38.551126,-90.335695,3.0,Local Services,Notaries,Mailbox Centers
1,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,"935 Race St, Philadelphia, PA 19107",80,19107,39.955505,-75.155564,4.0,Food,Bubble Tea,Coffee & Tea
2,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,"101 Walnut St, Green Lane, PA 18054",13,18054,40.338183,-75.471659,4.5,Breweries,Food,None
3,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,"615 S Main St, Ashland City, TN 37015",6,37015,36.269593,-87.058943,2.0,Fast Food,Sandwiches,Food
4,n_0UpQx1hsNbnPUSlodU8w,Famous Footwear,"8522 Eager Road, Dierbergs Brentwood Point, Br...",13,63144,38.627695,-90.340465,2.5,Fashion,Shoe Stores,Shopping
...,...,...,...,...,...,...,...,...,...,...,...
119693,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,"3388 Gateway Blvd, Edmonton, AB T6J 5H2",13,T6J 5H2,53.468419,-113.492054,3.0,Beauty & Spas,None,None
119694,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,"2813 Bransford Ave, Nashville, TN 37204",5,37204,36.115118,-86.766925,4.0,Nurseries & Gardening,Pet Stores,Hobby Shops
119695,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46, Indianapolis, IN 46250",8,46250,39.908707,-86.065088,3.5,Jewelry,Piercing,Toy Stores
119696,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,"2472 Troy Rd, Edwardsville, IL 62025",24,62025,38.782351,-89.950558,4.0,Eyewear & Opticians,Shopping,Sporting Goods


## Normalization With Area Codes

In [ ]:
restaurants.rename(columns={'postal_code': 'post'}, inplace=True)

In [ ]:
#The ZIP converter only has the first part of CA area codes,
#so shorten them
def truncate_canadian(entry):
    if entry.startswith('T'):
        return entry[:3]
    else:
        return entry
restaurants['post'] = restaurants['post'].apply(truncate_canadian)
#restaurants['post'][restaurants['post'].str.startswith('T')]

In [ ]:
restaurants['post'].astype(str).to_csv('postal.csv', index=False)

# Review Preprocessing

In [ ]:
#Drop reviews not about open restaurants
merged = reviews.merge(restaurants[['business_id']], on = 'business_id', how = 'right')
reviews = merged.dropna(subset = ['business_id'])

NameError: name 'reviews' is not defined

In [ ]:
reviews = reviews.reset_index()
reviews = reviews.drop('index', axis = 1)
reviews

# Map Preprocessing

In [ ]:
#Renaming
renames = {'X': 'longitude', 'Y': 'latitude', 'osm_id2': 'attraction_id', 'tourism': 'type', 'attraction_id': 'drop'}
t_map = t_map.rename(columns = renames)

In [ ]:
#Drop stuff
to_keep = ['longitude', 'latitude', 'website', 'addr_city', 'addr_country', 'addr_housenumber', 'addr_province', 'addr_postcode', 'addr_state', 'addr_street', 'addr_unit', 'name', 'type', 'attraction_id']
t_map = t_map[to_keep]

In [ ]:
#Unite addresses
feats = ['addr_housenumber', 'addr_street', 'addr_unit', 'addr_city', 'addr_state', 'addr_province', 'addr_country', 'addr_postcode']
for feat in feats:
  t_map[feat] = t_map[feat].astype(str)
  t_map[feat] = t_map[feat].replace('nan', '')

t_map['address'] = t_map['addr_housenumber'].str.strip() + ' ' + t_map['addr_street'].str.strip() + ', ' + t_map['addr_city'].str.strip() + ', ' + t_map['addr_state'].str.strip() + t_map['addr_province'].str.strip() + ' ' + t_map['addr_postcode'].str.strip() + ', ' + t_map['addr_country'].str.strip()


<ipython-input-7-f441aa9a97a1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_map[feat] = t_map[feat].astype(str)
<ipython-input-7-f441aa9a97a1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_map[feat] = t_map[feat].replace('nan', '')
<ipython-input-7-f441aa9a97a1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

In [ ]:
#Set unknown if any fields dropped
for index, row in t_map.iterrows():
  addr = row['address'].split(',')
  if any(field.strip() == '' for field in addr):
    t_map.at[index, 'address'] = 'Unknown'

In [ ]:
#Fill website column
t_map['website'].fillna('Unknown', inplace = True)
t_map['website']

0                                   Unknown
1                                   Unknown
2                                   Unknown
3         https://www.misionsurfmexico.com/
4                                   Unknown
                        ...                
265356                              Unknown
265357                              Unknown
265358                              Unknown
265359                              Unknown
265360                              Unknown
Name: website, Length: 265361, dtype: object

In [ ]:
#Second round drops
t_map = t_map.drop(feats, axis = 1)
t_map = t_map.dropna()

In [ ]:
t_map = t_map.reset_index(drop = True)
t_map

,longitude,latitude,website,name,type,attraction_id,address
0,-92.318096,14.616769,Unknown,Brisas del Mar,viewpoint,5327709923,Unknown
1,-92.355947,14.653835,Unknown,Hotel Playa Linda,hotel,388651468,Unknown
2,-92.240303,14.745767,Unknown,Rancho El Tesoro,camp_site,7883004685,Unknown
3,-92.433990,14.732903,https://www.misionsurfmexico.com/,Misión Surf Mexico,camp_site,7228473785,Unknown
4,-92.284821,14.865885,Unknown,Villas exotica,motel,4794499945,Unknown
...,...,...,...,...,...,...,...
126467,-74.716278,40.152368,Unknown,History,information,11770408378,Unknown
126468,-74.715997,40.152257,Unknown,History,information,11770408379,Unknown
126469,-74.714336,40.153047,Unknown,Deleware Canal Trail Map,information,11770408380,Unknown
126470,-94.206435,38.908874,Unknown,Fangorn Trail,information,11770424833,Unknown


# Distance Stuff

In [ ]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
rest_info = restaurants[['business_id', 'longitude', 'latitude']]
t_info = t_map[['attraction_id', 'longitude', 'latitude']]

In [ ]:
def haversine(lon1, lat1, lon2, lat2):

    #Radius of earth in km
    r = 6378.137

    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2

    c = 2 * np.arcsin(np.sqrt(a))
    dist = r * c
    return dist

In [ ]:
close_enough = 25 #kilometers

In [ ]:
gdf_rests = gpd.GeoDataFrame(rest_info, geometry=[Point(xy) for xy in zip(rest_info.longitude, rest_info.latitude)])
gdf_tours = gpd.GeoDataFrame(t_info, geometry=[Point(xy) for xy in zip(t_info.longitude, t_info.latitude)])

gdf_rests.crs = gdf_tours.crs = "EPSG:4326"

gdf_rests = gdf_rests.to_crs("EPSG:3857")
gdf_tours = gdf_tours.to_crs("EPSG:3857")

gdf_tours_buffered = gdf_tours.copy()
gdf_tours_buffered.geometry = gdf_tours_buffered.geometry.buffer(close_enough * 1000)

close_rests = gpd.sjoin(gdf_rests, gdf_tours_buffered, op='within')

close_rests_df = pd.DataFrame(close_rests.drop(columns='geometry')).reset_index(drop=True)

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [ ]:
close_rests_df = close_rests_df.drop(columns = 'index_right')
close_rests_df['distance'] = haversine(close_rests_df['longitude_left'], close_rests_df['latitude_left'], close_rests_df['longitude_right'], close_rests_df['latitude_right'])
close_rests_df = close_rests_df.drop(columns = ['longitude_left', 'latitude_left', 'longitude_right', 'latitude_right'])
close_rests_df

,business_id,attraction_id,distance
0,mpf3x-BjTdTEA3yCZrAYPw,4814607623,6.743928
1,n_0UpQx1hsNbnPUSlodU8w,4814607623,14.409151
2,pEm4xNCk8d0TF6A1gi-WDQ,4814607623,12.430846
3,VrGxoSNZfSiT-G4OqRRGIw,4814607623,13.743888
4,sB45WFgysT617bKWP_WJwA,4814607623,12.146948
...,...,...,...
17191985,g0fYqQRRKmYIfChE4jMLsg,5129580921,14.676975
17191986,g0fYqQRRKmYIfChE4jMLsg,366668005,11.434801
17191987,g0fYqQRRKmYIfChE4jMLsg,366668008,13.655961
17191988,g0fYqQRRKmYIfChE4jMLsg,10236738236,1.322636


# Long / Lat to ZIP

## ZIP-(Long, Lat) Converter

In [ ]:
def normalize_spacing(input_file, output_file):
    with open(input_file, 'r') as file:
        lines = file.readlines()

    normalized_lines = []
    for line in lines:
        try:
          parts = line.strip().split()
          postal_code = parts[1]
          latitude = parts[-3]
          longitude = parts[-2]
          latitude = float(latitude)
          longitude = float(longitude)

          normalized_line = f"{postal_code},{longitude},{latitude}"
          normalized_lines.append(normalized_line)
        except (ValueError, IndexError):
          pass

    with open(output_file, 'w') as file:
        file.write('\n'.join(normalized_lines))


In [ ]:
normalize_spacing('/content/drive/MyDrive/CSVs/ZIPRaws/CA.txt', 'canada.txt')
normalize_spacing('/content/drive/MyDrive/CSVs/ZIPRaws/US.txt', 'usa.txt')
normalize_spacing('/content/drive/MyDrive/CSVs/ZIPRaws/MX.txt', 'mexico.txt')

In [ ]:
canada = pd.read_csv('canada.txt', sep = ',', header = None)
usa = pd.read_csv('usa.txt', sep = ',', header = None)
mexico = pd.read_csv('mexico.txt', sep = ',', header = None)
converter = pd.concat([canada, usa, mexico], ignore_index = True)
converter.columns = ['post', 'longitude', 'latitude']

In [ ]:
94539 in converter['post']

True

In [ ]:
geo_converter = gpd.GeoDataFrame(converter, geometry = gpd.points_from_xy(converter['latitude'], converter['longitude']))
geo_converter = geo_converter[['post', 'geometry']]
geo_converter

,post,geometry
0,T0A,POINT (54.766 -111.717)
1,T0B,POINT (53.073 -111.582)
2,T0C,POINT (52.143 -111.694)
3,T0E,POINT (53.676 -115.095)
4,T0G,POINT (55.699 -114.453)
...,...,...
261611,99827,POINT (21.511 -103.288)
261612,99828,POINT (21.541 -103.226)
261613,99828,POINT (21.537 -103.206)
261614,99828,POINT (21.496 -103.232)


## Fill in attraction ZIPs

In [ ]:
geo_tourism = gpd.GeoDataFrame(t_map, geometry = gpd.points_from_xy(t_map['latitude'], t_map['longitude']))
geo_tourism = geo_tourism[['attraction_id', 'geometry']]
geo_tourism

,attraction_id,geometry
0,5327709923,POINT (14.617 -92.318)
1,388651468,POINT (14.654 -92.356)
2,7883004685,POINT (14.746 -92.240)
3,7228473785,POINT (14.733 -92.434)
4,4794499945,POINT (14.866 -92.285)
...,...,...
126467,11770408378,POINT (40.152 -74.716)
126468,11770408379,POINT (40.152 -74.716)
126469,11770408380,POINT (40.153 -74.714)
126470,11770424833,POINT (38.909 -94.206)


In [ ]:
from scipy.spatial import cKDTree

def ckdnearest(gdA, gdB):

    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    gdB_nearest = gdB.iloc[idx].drop(columns="geometry").reset_index(drop=True)
    gdf = pd.concat(
        [
            gdA.reset_index(drop=True),
            gdB_nearest,
            pd.Series(dist, name='dist')
        ],
        axis=1)

    return gdf

nearest = ckdnearest(geo_tourism, geo_converter)
nearest

,attraction_id,geometry,post,dist
0,5327709923,POINT (14.617 -92.318),30846,0.001222
1,388651468,POINT (14.654 -92.356),30835,0.032193
2,7883004685,POINT (14.746 -92.240),30851,0.022580
3,7228473785,POINT (14.733 -92.434),30830,0.000641
4,4794499945,POINT (14.866 -92.285),30785,0.002258
...,...,...,...,...
126467,11770408378,POINT (40.152 -74.716),8505,0.016029
126468,11770408379,POINT (40.152 -74.716),8505,0.015736
126469,11770408380,POINT (40.153 -74.714),8505,0.014932
126470,11770424833,POINT (38.909 -94.206),64070,0.048069


In [ ]:
nearest['post'].to_csv("tourposts.csv", index = False)

In [ ]:
#append to dataframe
t_map_zip = pd.concat([t_map, nearest[['post']]], axis = 1)
t_map_zip.to_csv('t_map.csv', index = False)

# Download Time

In [ ]:
reviews.to_csv('reviews.csv', index = False)

In [ ]:
restaurants.to_csv('restaurants.csv', index = False)

In [ ]:
close_rests_df.to_csv('close_rests.csv', index = False)

In [ ]:
t_map.to_csv('t_map.csv', index = False)

In [ ]:
reviews.to_csv('reviews.zip', index = False, compression='gzip')

# NLP Experiment

## Model Installation

In [ ]:
%pip install transformers[sentencepiece]

In [ ]:
#Model credit to Heng Yang:
#https://huggingface.co/yangheng/deberta-v3-base-absa-v1.1

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

absa_tokenizer = AutoTokenizer.from_pretrained("yangheng/deberta-v3-base-absa-v1.1")
absa_model = AutoModelForSequenceClassification \
  .from_pretrained("yangheng/deberta-v3-base-absa-v1.1")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/372 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

In [ ]:
from transformers import AutoModelForSequenceClassification

model_name = "yangheng/deberta-v3-base-absa-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

## Review Sampling Setup

In [ ]:
k = 25
mini = 300

tester = reviews.groupby('business_id', sort = False)
paired = reviews.groupby('business_id', sort = False).apply(lambda x: x.sample(n = min(k, len(x)))).reset_index(drop = True)
groups = paired.groupby('business_id', sort = False)
partition = [group for _, group in groups]

In [ ]:
def select_reasonable(lists, refer):
    keyed_lists = {}
    for l in lists:
        business_id = l['business_id'].iloc[0]
        if len(l) != 0 and len(refer.get_group(business_id)) >= mini:
            keyed_lists[business_id] = l
        else:
            keyed_lists[business_id] = None
    return keyed_lists

In [ ]:
usable = select_reasonable(partition, tester)

In [ ]:
len(usable)

9963

In [ ]:
len([ls for ls in usable.keys() if usable[ls] is not None])

677

## Use Model

In [ ]:
from tqdm import tqdm

In [ ]:
#Scoring parameters
attrs = ['food', 'drink', 'service', 'value']
results = {key: [] for key in attrs}

In [ ]:
#Define scoring metric based on predicted valence and confidence score
def adjust_score(d):
    if d['label'] == 'Positive':
        return d['score']
    elif d['label'] == 'Negative':
        return -d['score']
    else:
        return 0

In [ ]:
def score_grouped_small(idx, aspect):
  bid = restaurants['business_id'].iloc[idx]
  revs = usable[bid]
  if revs is not None:
    dicts = classifier(revs['text'].tolist(),  text_pair = aspect)
    labels = [adjust_score(d) for d in dicts]
    results[aspect].append(sum(labels))
  else:
    results[aspect].append(None)

In [ ]:
#Calculate + store score
#Switch aspect as needed

for i in tqdm(range(len(restaurants))):
    score_grouped_small(i, 'drink')

100%|██████████| 9963/9963 [1:23:32<00:00,  1.99it/s]


In [ ]:
drink_scores = pd.DataFrame(results['drink'], columns = ['drink'])
drink_scores.to_csv('results_drink.csv', index = False)

In [ ]:
value_scores = pd.DataFrame(results['value'], columns = ['value'])
value_scores.to_csv('results_value.csv', index = False)

In [ ]:
food_scores = pd.DataFrame(results['food'], columns = ['food'])
food_scores.to_csv('results_food.csv', index = False)

In [ ]:
service_scores = pd.DataFrame(results['service'], columns = ['service'])
service_scores.to_csv('results_service.csv', index = False)

# Finishing Up

In [ ]:
drinks = pd.read_csv('/content/drive/MyDrive/Scoresheets/results_drink.csv')
values = pd.read_csv('/content/drive/MyDrive/Scoresheets/results_value.csv')
foods = pd.read_csv('/content/drive/MyDrive/Scoresheets/results_food.csv')
services = pd.read_csv('/content/drive/MyDrive/Scoresheets/results_service.csv')

In [ ]:
total = pd.concat([drinks, values, foods, services], axis = 1)
total

In [ ]:
total.to_csv('scores.csv', index = False)

# ZIP Code Lookup


In [ ]:
restcodes = restaurants['post']
attcodes = nearest['post']

In [ ]:
unique_zips = pd.concat([restcodes, attcodes], axis = 0).unique()
df = pd.DataFrame({'post': unique_zips})
converter.post = converter.post.astype(str)
df.to_csv('unique_zips.csv', index=False)

In [ ]:
df.post = df.post.str.strip()
converter.post = converter.post.str.strip()
merged = pd.merge(df, converter, left_on = 'post', right_on = 'post')
merged.post = merged.post.astype(str)
merged.to_csv('longlat_lookup', index = False)

In [ ]:
merged

,post,longitude,latitude
0,63123,-90.3241,38.5476
1,63123,-90.3241,38.5476
2,63123,-90.3241,38.5476
3,63123,-90.3241,38.5476
4,19107,-75.1593,39.9487
...,...,...,...
5515,E8S,-64.7247,47.7287
5516,L3T,-79.3946,43.8227
5517,M8Z,-79.5231,43.6256
5518,H1H,-73.6389,45.5899
